##### 问题:
我们想在访问实例的属性时能够将其委托（delegate）到一个内部持有的对象上，这可
以作为继承的替代方案或者是为了实现一种代理机制。

##### 解决方案：
简单地说，委托是一种编程模式。我们将某个特定的操作转交给（委托）另一个不同
的对象实现。通常来说，最简单的委托形式看起来是这样的：

In [31]:
class A:
    def spam(self, x):
        pass
    def foo(self):
        pass
class B:
    def __init__(self):
        self._a = A()  #创建一个A的实例
    def spam(self, x):
    # Delegate to the internal self._a instance
        return self._a.spam(x)
    def foo(self):
    # Delegate to the internal self._a instance
        return self._a.foo()
    def bar(self):
        pass 


如果仅有几个方法需要委托，编写像上面那样的代码是非常简单的。但是，如果有许
多方法都需要委托，另一种实现方式是定义__getattr__()方法，就像下面这样：

In [32]:
class A:
    def spam(self, x):
        pass
    def foo(self):
        pass
class B:
    def __init__(self):
        self._a = A()
    def bar(self):
        pass
    # Expose all of the methods defined on class A
    def __getattr__(self, name):
        return getattr(self._a, name) 

\_\_getattr\_\_()方法能用来查找所有的属性。如果代码中尝试访问一个并不存在的属性，
就会调用这个方法。在上面的代码中，我们在访问 B 中未定义的方法时就能把这个操
作委托给 A。示例如下：

In [33]:
b = B()
b.bar()
b.spam(42) 

委托的另一个例子就是在实现代理时。示例如下：

In [34]:
class Proxy:
    def __init__(self, obj):
        self._obj = obj
    # Delegate attribute lookup to internal obj
    def __getattr__(self, name):
        print('getattr:', name)
        return getattr(self._obj, name)
    # Delegate attribute assignment
    def __setattr__(self, name, value):
        if name.startswith('_'):
            super().__setattr__(name, value)
        else:
            print('setattr:', name, value)
            setattr(self._obj, name, value)
    # Delegate attribute deletion
    def __delattr__(self, name):
        if name.startswith('_'):
            super().__delattr__(name)
        else:
            print('delattr:', name)
            delattr(self._obj, name)

要使用这个代理类，只要简单地用它包装另一个实例即可。示例如下：

In [35]:
class Spam:
    def __init__(self, x):
        self.x = x
    def bar(self, y):
        print('Spam.bar:', self.x, y)
# Create an instance
s = Spam(2)
# Create a proxy around it
p = Proxy(s)
# Access the proxy
print(p.x) # Outputs 2
p.bar(3) # Outputs "Spam.bar: 2 3"
p.x = 37 # Changes s.x to 37

getattr: x
2
getattr: bar
Spam.bar: 2 3
setattr: x 37


通过自定义实现属性的访问方法，就可以对代理进行定制化处理，让其表现出不同的
行为（例如，访问日志、只允许只读访问等）。

委托有时候可以作为继承的替代方案。例如，不要编写下面这样的代码：

In [36]:
class A:
    def spam(self, x):
        print('A.spam', x)
    def foo(self):
        print('A.foo')
class B(A):
    def spam(self, x):
        print('B.spam')
        super().spam(x)
    def bar(self):
        print('B.bar')

用到了委托的实现方案则会是这样：

In [37]:
class A:
    def spam(self, x):
        print('A.spam', x)
    def foo(self):
        print('A.foo')

class B:
    def __init__(self):
        self._a = A()   #直接实例化A,发挥继承的效果
    def spam(self, x):
        print('B.spam', x)
        self._a.spam(x)
    def bar(self):
        print('B.bar')
    def __getattr__(self, name):
        return getattr(self._a, name)

有时候当直接使用继承可能没多大意义，或者我们想更多地控制对象之间的关系（例
如只暴露出特定的方法、实现接口等），此时使用委托会很有用。

当使用委托来实现代理时，这里还有几个细节需要注意。首先，\_\_getattr\_\_()实际上是
一个回滚（fallback）方法，它只会在某个属性没有找到的时候才会调用。因此，如果
访问的是代理实例本身的属性（例如本例中的_obj 属性），这个方法就不会被触发调
用。其次，__setattr__()和__delattr__()方法需要添加一点额外的逻辑来区分代理实例本
身的属性和内部对象_obj 上的属性。常用的惯例是代理类只委托那些不以下划线开头
的属性（即，代理类只暴露内部对象中的“公有”属性）。

同样需要重点强调的是__getattr__()方法通常不适用于大部分名称以双下划线开头和结
尾的特殊方法。例如，考虑下面这个类：

In [38]:
class ListLike:
    def __init__(self):
        self._items = []
    def __getattr__(self, name):
        return getattr(self._items, name)

如果尝试创建一个 ListLike 对象，就会发现它能支持常见的列表方法，例如 append()
和 insert()。但是，却无法支持 len()、查找元素等操作。示例如下：

In [39]:
a = ListLike()
a.append(2)
a.insert(0, 1)
a.sort() 
#a[0]  # TypeError: 'ListLike' object is not subscriptable

要支持不同的操作，必须自行手动委托相应的特殊方法。示例如下：

In [ ]:
class ListLike:
    def __init__(self):
        self._items = []
    def __getattr__(self, name):
        return getattr(self._items, name)
    # Added special methods to support certain list operations
    def __len__(self):  #增加len方法
        return len(self._items)
    def __getitem__(self, index): #增加index并访问
        return self._items[index]
    def __setitem__(self, index, value): #增加index并修改
        self._items[index] = value
    def __delitem__(self, index):  #增加index并删除
        del self._items[index] 